In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
from sklearn.utils import shuffle


imdb = pd.read_csv('../Sup/IMDB Dataset.csv')
imdb = shuffle(imdb)
imdb.head()
train_X, test_X = np.array(imdb.filtered_review[:45000]),np.array(imdb.filtered_review[45000:])
train_y, test_y = np.array(imdb.sentiment[:45000]),np.array(imdb.sentiment[45000:])

In [3]:
vocab_size = 10000
embedding_dim = 10
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
tokenizer = Tokenizer(vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_X)
train_X = pad_sequences(sequences,max_length,truncating='post',padding='post')

test_seq = tokenizer.texts_to_sequences(test_X)
test_X = pad_sequences(test_seq,max_length,padding='post',truncating='post')

In [6]:
def binarizing(a):
    if a == 'positive':
        return 1
    else:
        return 0
bina = np.vectorize(binarizing)

In [7]:
train_y = bina(train_y)
test_y = bina(test_y)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 10)           100000    
_________________________________________________________________
flatten (Flatten)            (None, 1200)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 7206      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 107,213
Trainable params: 107,213
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
model.fit(train_X,train_y,batch_size=1000,
          epochs=10,verbose=1,
          validation_data=(test_X,test_y))

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 2s 37us/sample - loss: 0.6902 - accuracy: 0.5429 - val_loss: 0.6754 - val_accuracy: 0.6498

In [11]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

(10000, 10)


In [12]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [13]:
import os

model_save_path = '../Sup'
weight_save_path = '../Sup'
if not os.path.exists(model_save_path):
    os.mkdir(model_save_path)
if not os.path.exists(weight_save_path):
    os.mkdir(weight_save_path)
model.save( '../Sup/sentimental_model.1.h5')
model.save_weights( "../Sup/sentimental_model_weights1.1.h5")

In [14]:
import pickle

with open('../Sup/sentimental_1_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)